In [1]:
import sys; print(sys.executable)

/Users/huac01/Documents/Code/project/myenv-311/bin/python


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator, FuncFormatter
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import linear_sum_assignment
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.impute import KNNImputer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, pairwise_distances
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
import umap
import igraph as ig
import leidenalg as la
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D 

pd.set_option("display.max_columns", None)  # None = no limit for when i do df.head()

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys, os
scripts_dir = os.path.abspath("../scripts")  # note the ../
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)


In [4]:
from functions_needed_for_pipeline import (calculate_pol2_position, 
    preprocess_dataframe, 
    filter_reads_per_gene_middle_bin_name, 
    bin_then_matrix, 
    process_into_matrix, 
    handling_NaN, 
    preprocess_long_for_plot, 
    plot_reads_long, 
    get_genes_list, 
    get_groups_df_list,
    clustering_final, 
    compute_cluster_metrics,
    plot_umap, 
    dict_id_cluster_color, 
    dict_to_df, 
    merge, 
    start_end_center, 
    summary, 
    compute_bulk_centroids, 
    compute_gene_centroids, 
    plot_centroids_with_shading,
    build_positional_weights
)

In [5]:
##LIST CONTAINITNG THE GROUPS NAMES 
treatment_groups = [
        'BT474_mV_72hNHWD',
        'BT474_mV_72hSTC15',
        'BT474_mV_high_Untreated',
        'BT474_mV_low_Untreated',
        'BT474_mV_Untreated_Unsorted']

## ARBITRARY CHOSEN COLORS FOR THE CLUSTERS (THEY NEED TO HAVE A HIGH VALUE AND CHROMA TO STAND OUT ON A GREY BACKGROUND)
hex_colors = [
    "#0e67a7", "#ff7f0e", "#a0e468", "#d62728", "#9467bd",
    "#672417", "#e377c2", "#f5f523", "#28e0f5", "#3214a8",
    "#ca9d16", "#04a887", "#8c564b", "#17becf", "#bcbd22",
    "#2ca02c", "#1f77b4", "#ff9896", "#c5b0d5", "#98df8a",
    "#ffbb78", "#aec7e8", "#7f7f7f", "#c49c94", "#dbdb8d",
    "#9edae5"
]

In [6]:
with open("../data/gc_peak2_all_colnames.txt", "r") as f:
    description_colnames = f.read()

print(description_colnames)  

readid
chr: chr with "chr"
motif_stand : gene orientation +/-
gene : can have duplicates if the gene has isoforms (multiple TSS)
group : conditions: BT474_mV_72hNHWD : pretreated with NHWD (are in G0)
                  BT474_mV_72hSTC15  : pretreated with STC  (are in G0)
                  BT474_mV_high_Untreated : basleine G0 cells
                  BT474_mV_low_Untreated : cycling cells
                  BT474_mV_Untreated_Unsorted : unsorted, untreated (cycling + G0)
tss_pos : gene start , use with motif_stand now were is upstream vs downstream
pol2: pol2/nopol2 (peak or not) : in this set everything is pol2
cov: number of read overlapping the site 
C_start : GpC start
C_end : GpC end (you can just use C_end)
C_pos :  GpC position relative to pol2 summit
meth: GpC meth status 0/1
motif_center: pol2 summit coordinate
peakid: pol2 peak id
min  : relative lowest upstream GpC position on the read
max :   relative max upstream GpC position on the read
range : read range (in bp)



In [7]:
df = pd.read_csv("../data/gch_Hela_S3_G1_CTCF_peaks_formatted.txt", delim_whitespace=True)

print(f'The shape of the dataframe is of {df.shape}')
#print(df.dtypes)
#print(df.columns)

df.head()

The shape of the dataframe is of (170894882, 29)


,chr,motif_start,motif_end,pol2_summit,readid,read_strand,group,smc3,h3k27ac,cov,C_start,C_end,meth,N,motif_center,C_pos,C_in_motif,peakid,meth_C_in_motif,read_meth_C_in_motif,n_meth_motif,perc_meth_in_motif,min,max,range,peak_cov,peak_meth,n,read_meth_mean
0,chr9,122930263,122932265,122932954,0000016a-0de6-41bc-afd6-0d719e7fe594,OT,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,59,122931178,122931179,0,1167,122931264.0,-85.0,1,chr9:122930263-122932265,0,1,1286.0,2179.7,-25011.0,11802.0,36813,59.000000,0.21048,1.000000,0.333333
1,chr9,122930510,122932512,122932954,0000016a-0de6-41bc-afd6-0d719e7fe594,OT,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,61,122932600,122932601,1,1167,122931511.0,1090.0,0,chr9:122930510-122932512,1,606,993.4,-25599.0,10453.0,36052.0,61,0.291876,1.00000,0.333333,NaN
2,chr9,122938417,122940419,122940191,0000016a-0de6-41bc-afd6-0d719e7fe594,OT,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,76,122931225,122931226,0,1167,122939418.0,-8192.0,0,chr9:122938417-122940419,1,324,426.3,-36192.0,3648.0,39840.0,76,0.276115,1.00000,0.333333,NaN
3,chr1,151346204,151348206,151348049,00000644-0dc2-4348-bba5-fa2e8c5c25f5,OT,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,1,1,92,151348763,151348764,0,342,151347205.0,1559.0,0,chr1:151346204-151348206,1,90,97.8,-92462.0,68231.0,160693.0,92,0.200361,2.00000,0.250000,NaN
4,chr1,151346019,151348021,151348049,00000644-0dc2-4348-bba5-fa2e8c5c25f5,OT,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,1,1,94,151348194,151348195,0,342,151347020.0,1175.0,0,chr1:151346019-151348021,1,9,9.6,-89719.0,27554.0,117273.0,94,0.212766,2.00000,0.250000,NaN


In [8]:
print(df.columns)

Index(['chr', 'motif_start', 'motif_end', 'pol2_summit', 'readid',
       'read_strand', 'group', 'smc3', 'h3k27ac', 'cov', 'C_start', 'C_end',
       'meth', 'N', 'motif_center', 'C_pos', 'C_in_motif', 'peakid',
       'meth_C_in_motif', 'read_meth_C_in_motif', 'n_meth_motif',
       'perc_meth_in_motif', 'min', 'max', 'range', 'peak_cov', 'peak_meth',
       'n', 'read_meth_mean'],
      dtype='object')


In [9]:
##RUNNING THE CODE ON THE ACTUAL DATASET -- applying preprocess_dataframe
columns_to_drop=['read_strand', 'N', 'n_meth_motif',
       'perc_meth_in_motif','peak_cov', 'peak_meth', 'C_in_motif',
       'n', 'read_meth_mean', 
       # 'motif_id', 'gene2', 'bound',
        'cov', 'peakid', 'meth_C_in_motif','read_meth_C_in_motif']

df_proc = preprocess_dataframe(df, columns_to_drop)

df_proc.head()

'pol2' column not found; skipping conversion to binary marker.
Gene_tss created from readid and C_start columns
Final shape: (54224141, 18)
NUmber of gene_tss: 53320882


,chr,motif_start,motif_end,pol2_summit,readid,group,smc3,h3k27ac,C_start,C_end,meth,motif_center,C_pos,min,max,range,pol2_pos,gene_tss
0,9.0,122930263,122932265,122932954,0000016a-0de6-41bc-afd6-0d719e7fe594,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,122931178,122931179,0,122931264.0,-85.0,-25011.0,11802.0,36813,122931264,0000016a-0de6-41bc-afd6-0d719e7fe594_122931178
1,9.0,122930510,122932512,122932954,0000016a-0de6-41bc-afd6-0d719e7fe594,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,122932600,122932601,1,122931511.0,1090.0,10453.0,36052.0,61,122931511,0000016a-0de6-41bc-afd6-0d719e7fe594_122932600
2,9.0,122938417,122940419,122940191,0000016a-0de6-41bc-afd6-0d719e7fe594,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,0,1,122931225,122931226,0,122939418.0,-8192.0,3648.0,39840.0,76,122939418,0000016a-0de6-41bc-afd6-0d719e7fe594_122931225
3,1.0,151346204,151348206,151348049,00000644-0dc2-4348-bba5-fa2e8c5c25f5,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,1,1,151348763,151348764,0,151347205.0,1559.0,68231.0,160693.0,92,151347205,00000644-0dc2-4348-bba5-fa2e8c5c25f5_151348763
4,1.0,151346019,151348021,151348049,00000644-0dc2-4348-bba5-fa2e8c5c25f5,TSS_PolII_noSororin_SMC3_H3K4me3_no_CTCF_SG2,1,1,151348194,151348195,0,151347020.0,1175.0,27554.0,117273.0,94,151347020,00000644-0dc2-4348-bba5-fa2e8c5c25f5_151348194


In [10]:
# WHOLE PREPROCESSED DF THAT GETS BINNED UNDER METHYLATION MATRIX (i need it to flter the genes that we want to work on)

df_whole_matrix= process_into_matrix(
    df_proc,
    gene=None, #the id taken from the column gene_tss, only if we want to process per gene
    bin_size= 50)

df_whole_matrix.head()

: 

In [ ]:
# FILTERING AND KEEPING ONLY THE GENES THAT ARE SUITABLE (HAVE ENOUGH READS OVERLAPPING AND LONG ENOUGH)

df_filtered, filtered_regions = filter_reads_per_gene_middle_bin_name(
    df_whole_matrix, 
    middle_bin_name=None, 
    min_reads =50, 
    min_bins =40,
    require_middle_bin=True)

print(f'THe shape of the filtered dataframe is of {df_filtered.shape}')
print(f'There is {df_filtered.index.get_level_values("gene_tss").nunique()} different genes in this dataframe')

df_filtered.head()

THe shape of the filtered dataframe is of (13485, 41)
There is 70 different genes in this dataframe


bin                                                                                                   -1000  \
gene_tss                 readid                               group                       cluster             
ABCC11_48244588_48244588 0233446d-0980-4045-a62a-c46635a83547 BT474_mV_Untreated_Unsorted 7        0.000000   
                         051d8420-2d91-4c64-8ec5-dd07b0c59c69 BT474_mV_Untreated_Unsorted 4             NaN   
                         058844d4-c062-4447-bd3f-6d5198529778 BT474_mV_Untreated_Unsorted 12       0.000000   
                         091e0971-e6a2-4151-8f1d-f88e807e268d BT474_mV_Untreated_Unsorted 11            NaN   
                         0aaab132-3f49-4aa2-8db2-11b1f7da90a9 BT474_mV_72hNHWD            4        0.666667   

bin                                                                                                   -950   \
gene_tss                 readid                               group                       cluster             
ABCC11_48244588_48244588 0233446d-0980-4045-a62a-c46635a83547 BT474_mV_Untreated_Unsorted 7        1.000000   
                         051d8420-2d91-4c64-8ec5-dd07b0c59c69 BT474_mV_Untreated_Unsorted 4             NaN   
                         058844d4-c062-4447-bd3f-6d5198529778 BT474_mV_Untreated_Unsorted 12       0.000000   
                         091e0971-e6a2-4151-8f1d-f88e807e268d BT474_mV_Untreated_Unsorted 11            NaN   
                         0aaab132-3f49-4aa2-8db2-11b1f7da90a9 BT474_mV_72hNHWD            4        0.666667   

bin                                                                                                -900   \
gene_tss                 readid                               group                       cluster          
ABCC11_48244588_48244588 0233446d-0980-4045-a62a-c46635a83547 BT474_mV_Untreated_Unsorted 7         0.00   
                         051d8420-2d91-4c64-8ec5-dd07b0c59c69 BT474_mV_Untreated_Unsorted 4          NaN   
                         058844d4-c062-4447-bd3f-6d5198529778 BT474_mV_Untreated_Unsorted 12        0.25   
                         091e0971-e6a2-4151-8f1d-f88e807e268d BT474_mV_Untreated_Unsorted 11         NaN   
                         0aaab132-3f49-4aa2-8db2-11b1f7da90a9 BT474_mV_72hNHWD            4         0.25   

bin                                                                                                -850   \
gene_tss                 readid                               group                       cluster          
ABCC11_48244588_48244588 0233446d-0980-4045-a62a-c46635a83547 BT474_mV_Untreated_Unsorted 7          0.0   
                         051d8420-2d91-4c64-8ec5-dd07b0c59c69 BT474_mV_Untreated_Unsorted 4          NaN   
                         058844d4-c062-4447-bd3f-6d5198529778 BT474_mV_Untreated_Unsorted 12         0.0   
                         091e0971-e6a2-4151-8f1d-f88e807e268d BT474_mV_Untreated_Unsorted 11         NaN   
                         0aaab132-3f49-4aa2-8db2-11b1f7da90a9 BT474_mV_72hNHWD            4          0.0   

bin                                                                                                -800   \
gene_tss                 readid                               group                       cluster          
ABCC11_48244588_48244588 0233446d-0980-4045-a62a-c46635a83547 BT474_mV_Untreated_Unsorted 7          0.5   
                         051d8420-2d91-4c64-8ec5-dd07b0c59c69 BT474_mV_Untreated_Unsorted 4          NaN   
                         058844d4-c062-4447-bd3f-6d5198529778 BT474_mV_Untreated_Unsorted 12         0.0   
                         091e0971-e6a2-4151-8f1d-f88e807e268d BT474_mV_Untreated_Unsorted 11         NaN   
                         0aaab132-3f49-4aa2-8db2-11b1f7da90a9 BT474_mV_72hNHWD            4          0.0   

bin                                                                                                -750   \
gene_tss                 readid                               gro